In [12]:
from assets.utils.txt import read_txt
import pandas as pd

row = read_txt('./data/custom_movie_rating.txt')
column_names = ["CustomID", "MovieID", "MovieName", "Rating"]
custom_movie_rating_df = pd.DataFrame(row, columns=column_names)
custom_movie_rating_df

,CustomID,MovieID,MovieName,Rating
0,ZBm5R0Mj7qd46,m5XKVDL,양자물리학,3.0\n
1,ZBm5R0Mj7qd46,m5Nn2GE,영화로운 작음,3.5\n
2,ZBm5R0Mj7qd46,mWvq0vZ,내 방,2.5\n
3,ZBm5R0Mj7qd46,mdKpyBk,성적표의 김민영,3.0\n
4,ZBm5R0Mj7qd46,mOopbwm,킹덤4: 대장군의 귀환,3.0\n
...,...,...,...,...
3068,ZBm5R0Mj7qd46,m5ZXrxO,아저씨,4.0\n
3069,ZBm5R0Mj7qd46,mOPolaW,13층,3.0\n
3070,ZBm5R0Mj7qd46,m5XMeBW,아이언맨,4.0\n
3071,ZBm5R0Mj7qd46,m5GXE2W,닌자 어쌔신,3.0\n


In [22]:
row = read_txt('./data/movie_info_watcha.txt')
column_names = ["MovieID", "Title", "Year", "Genre", "Country", "Runtime", "Age", "Cast_Production_Info_List", "Synopsis", "Avg_Rating", "N_Rating(만명)","N_Comments"]
movie_info = pd.DataFrame(row, columns=column_names)

In [32]:
movie_info[movie_info['Title'] == 'None']

,MovieID,Title,Year,Genre,Country,Runtime,Age,Cast_Production_Info_List,Synopsis,Avg_Rating,N_Rating(만명),N_Comments
44,m5YrbQW,None,None,None,None,None,None,[],None,None,None,None\n
45,mOoyZZ5,None,None,None,None,None,None,[],None,None,None,None\n
46,m5e8v3d,None,None,None,None,None,None,[],None,None,None,None\n
47,m1d62gd,None,None,None,None,None,None,[],None,None,None,None\n
48,mdBzJbk,None,None,None,None,None,None,[],None,None,None,None\n
...,...,...,...,...,...,...,...,...,...,...,...,...
13783,mJOVK8d,None,None,None,None,None,None,[],None,None,None,None\n
13784,mWvkl6b,None,None,None,None,None,None,[],None,None,None,None\n
13785,mdKMw0V,None,None,None,None,None,None,[],None,None,None,None\n
13786,m5rXjbO,None,None,None,None,None,None,[],None,None,None,None\n


In [14]:
column_names = ["MovieID", "CustomID", "Comment", "Rating", "N_Likes"]
row = read_txt("./data/movie_comments.txt")

# 각 청크를 병합하여 데이터프레임 생성
movie_comments_df = pd.DataFrame(row, columns=column_names)
movie_comments_df

,MovieID,CustomID,Comment,Rating,N_Likes
0,m5XKVDL,yKZx3nJg254dJ,"""양자물리학""이라는 제목만 빼고 전반적으로 다 괜찮은 영화. . . #현시점대한민국...",3.0,152\n
1,m5XKVDL,nkPvrb8wrvgar,"안녕하세요, 물리학과입니다. 이런게 물리라면 저는 문과입니다. Never quite...",2.0,141\n
2,m5XKVDL,yKZx37WjDx4dJ,양자물리학을 억지로 개연성도 없이 영화에 삽입시키고있다 검사님 연기 리스펙,2.5,128\n
3,m5XKVDL,6ewxakKkyvQ1m,검사 연기가 후덜덜 특히 곽철용과의 씬들이 재밌는데 거의 검사실 몰카 보는 수준이다...,4.0,97\n
4,m5XKVDL,Qgy51rmX7xDjk,우아는커녕 추잡스럽게.,2.0,91\n
...,...,...,...,...,...
17043,m5xJj6x,86ADvGyLgxzZl,"사회의 편견, 대중들의 (관심의 탈을 쓴)무례, 물리적 시간의 흐름, (대중)예술론...",3.5,5\n
17044,m5xJj6x,8nPvy7YdXkxYo,[소리의 괴성] 신인감독 문소리를 응원합니다,5.0,5\n
17045,m5xJj6x,87Gv7wEa3vE6o,상처로 콕콕 박혔을 말들을 대사로 만들어 웃음으로 돌려주는 놀라움. 고난을 웃음으로...,4.0,5\n
17046,m5xJj6x,87Gv7mPGjxE6o,본인의 역을 연기하는 기분은 어떤 기분일까. 정말 대본대로 짜여진 스토리인지 본인의...,4.0,5\n


In [17]:
set(movie_comments_df['CustomID'])

{'RE952BmMn5Q72',
 'NP9vL2dMWjx6k',
 'nb4xkBQyM5OAz',
 'g64qzAJbMxER0',
 'JZ4vBkJMm5RXO',
 'VRZv44O1evr6y',
 '2mwvgLwV45Ma7',
 'zNM5NYnR7nv26',
 'YMKqmOz6r5loD',
 '87Gv77BOmvE6o',
 'JgAx8bW6DvLbO',
 'jae5WOBaK51P3',
 'DgwxADjyeM5rM',
 'g64qz3PlGgvER',
 'Mr95neJjBvZPG',
 'jae5W4BgYx1P3',
 'a9L5P4w9eO5wg',
 'nb4xkY2DdvOAz',
 '87Gv7zNVJqE6o',
 'Pznx9e64jx761',
 '6ADvGQlpdxzZl',
 'Mr95nnQR75ZPG',
 'Qgy51pQoaXvDj',
 'JgkqlJm2Km5X0',
 'YaR5YNkKpxgX1',
 'ZWwRqoomWqlzB',
 'VRZv4nDObxr6y',
 'dP8v6PwkQxWeJ',
 'Pznx9mZYMgx76',
 'yKZx3jeWb54dJ',
 'Pznx9Kmk2Zv76',
 'R6OvKWen0jvN8',
 '4WLxZzaVkxroA',
 'K6ExjjM9zxX1O',
 '6ewxaPG6Z5Q1m',
 'ld0q0GQWPx6Xn',
 '8nPvygpaoxYo0',
 'Qgy51wgEDqDjk',
 'RE952dnJGaqQ7',
 'YMKqmkazEE5lo',
 'zNM5NgRyl4x26',
 '6ADvGQdalgxzZ',
 'K6ExjoZ0VqX1O',
 'Mr95nNdOk5ZPG',
 '3Bnvw0yaRR5PA',
 'ld0q0YK2VGv6X',
 'Pznx9D6rj5761',
 'dP8v6okJ85WeJ',
 '7ld0q0y0Ev6Xn',
 'nkPvrJRDPDvga',
 'zNM5NrKpK526j',
 'jae5WJlO96v1P',
 'YMKqmNY3GvloD',
 'a9L5PmwEPxwg8',
 'OkexJZgjjxdbw',
 '17ovVkNd

In [35]:
input_file = './data/movie_info_watcha.txt'
output_file = './data/movie_info_watcha_cleaned.txt'

# 파일 읽고 조건에 따라 줄 삭제
with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    for line_number, line in enumerate(infile, start=1):
        try:
            # 조건: 두 번째 필드가 'None'인 경우 해당 줄 건너뜀
            if line.split('/')[1] != 'None':
                outfile.write(line)  # 조건에 맞지 않으면 줄을 그대로 저장
        except IndexError:
            # 줄이 잘못된 경우 (예: 구분자가 부족한 경우)도 건너뜀
            continue

print(f"Filtered file saved to {output_file}")


Filtered file saved to ./data/movie_info_watcha_cleaned.txt
